In [2]:
import csv
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM, Dropout,Embedding, Bidirectional
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('arabic'))

[nltk_data] Downloading package stopwords to C:\Users\Ahmed
[nltk_data]     Sherif\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [3]:
vocab_size = 5000
embedding_dim = 64
max_length = 200
oov_tok = '<OOV>' #  Out of Vocabulary
training_portion = 0.8

In [4]:
articles = []
labels = []

with open("Data/arabic_main.csv", 'r',encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        labels.append(row[1])
        article = row[0]
        for word in STOPWORDS:
            token = ' ' + word + ' '
            article = article.replace(token, ' ')
            article = article.replace(' ', ' ')
        articles.append(article)

In [5]:
len(labels),len(articles)

(7991, 7991)

In [6]:
print(labels[0])

schedule


In [7]:
train_size = int(len(articles) * training_portion)

train_articles = articles[0: train_size]
train_labels = labels[0: train_size]

validation_articles = articles[train_size:]
validation_labels = labels[train_size:]

In [8]:
print('train_articles' ,len(train_articles))
print('train_labels', len(train_labels))
print('validation_articles', len(validation_articles))
print('validation_labels', len(validation_labels))

train_articles 6392
train_labels 6392
validation_articles 1599
validation_labels 1599


In [9]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_articles)
word_index = tokenizer.word_index

In [10]:
import pickle
filename = f'../../utils/tokenizer.sav'
pickle.dump(tokenizer, open(filename, 'wb'))

In [11]:
word_index

{'<OOV>': 1,
 'هل': 2,
 'ما': 3,
 'الطقس': 4,
 'الساعة': 5,
 '،': 6,
 'الخاص': 7,
 'سيكون': 8,
 'يوم': 9,
 'يمكنك': 10,
 'مساءً': 11,
 'التقويم': 12,
 'موعد': 13,
 'اليوم': 14,
 'الأسبوع': 15,
 'كيف': 16,
 'التحقق': 17,
 'فضلك': 18,
 'وقت': 19,
 'من': 20,
 'توقعات': 21,
 'أريد': 22,
 'ستكون': 23,
 'الحدث': 24,
 'ماذا': 25,
 '30': 26,
 'الجو': 27,
 'بي': 28,
 'موعدًا': 29,
 'حدد': 30,
 'لدي': 31,
 'أود': 32,
 'طبيب': 33,
 'إضافة': 34,
 'مارس؟': 35,
 'مارس': 36,
 'قل': 37,
 'أعرف': 38,
 'يمكن': 39,
 '3': 40,
 '4': 41,
 'تاريخ': 42,
 'يرجى': 43,
 'أحتاج': 44,
 'اليوم؟': 45,
 'الشهر': 46,
 'صباحًا': 47,
 'متى': 48,
 'تخبرني': 49,
 'الأحداث': 50,
 '5': 51,
 'ليوم': 52,
 'المقبل': 53,
 'الظهر': 54,
 'القادم': 55,
 'درجة': 56,
 'نعم': 57,
 'كم': 58,
 '11': 59,
 'شكرا': 60,
 'في': 61,
 'الحياة': 62,
 'تحقق': 63,
 'أخبرني': 64,
 'أكون': 65,
 '15': 66,
 '12': 67,
 'منبه': 68,
 'ووقت': 69,
 'عندما': 70,
 'الثاني': 71,
 'الوقت': 72,
 'شكرًا': 73,
 'بي؟': 74,
 'يكون': 75,
 '10': 76,
 'يجب': 77,
 '2

In [12]:
tokenizer.texts_to_sequences(['what will be the weather tommorrow morning ?'])

[[1, 1, 1, 1193, 1, 1, 1]]

In [13]:
train_sequences = tokenizer.texts_to_sequences(train_articles)
len(train_sequences)

6392

In [14]:
train_sequences[0]

[30, 29, 223, 139, 14, 5, 40, 11, 6, 600, 2358, 730, 1259, 2359]

In [15]:
train_padded = pad_sequences(train_sequences, maxlen=max_length,)

In [16]:
len(train_padded),len(train_padded[0])

(6392, 200)

In [17]:
validation_sequences = tokenizer.texts_to_sequences(validation_articles)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length)

In [18]:
set(labels)

{'general', 'greeting', 'recommendation', 'schedule', 'thank', 'weather'}

In [19]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(validation_labels))

In [20]:
validation_label_seq

array([[3],
       [3],
       [1],
       ...,
       [3],
       [5],
       [1]])

In [21]:
# train_labels[744], training_label_seq[744]

In [22]:
model = Sequential()
model.add(Embedding(vocab_size,embedding_dim ))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(embedding_dim)))
model.add(Dense(7,activation='softmax' ))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          320000    
                                                                 
 dropout (Dropout)           (None, None, 64)          0         
                                                                 
 bidirectional (Bidirectiona  (None, 128)              66048     
 l)                                                              
                                                                 
 dense (Dense)               (None, 7)                 903       
                                                                 
Total params: 386,951
Trainable params: 386,951
Non-trainable params: 0
_________________________________________________________________


In [23]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(loss='sparse_categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

c:\Users\Ahmed Sherif\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [24]:
num_epochs = 10
history = model.fit(train_padded, training_label_seq, epochs=num_epochs, validation_data=(validation_padded, validation_label_seq), verbose=2)

Epoch 1/10
200/200 - 69s - loss: 0.8889 - accuracy: 0.6802 - val_loss: 0.3615 - val_accuracy: 0.9099 - 69s/epoch - 344ms/step
Epoch 2/10
200/200 - 59s - loss: 0.2426 - accuracy: 0.9327 - val_loss: 0.2014 - val_accuracy: 0.9468 - 59s/epoch - 295ms/step
Epoch 3/10
200/200 - 57s - loss: 0.1294 - accuracy: 0.9651 - val_loss: 0.1689 - val_accuracy: 0.9475 - 57s/epoch - 284ms/step
Epoch 4/10
200/200 - 54s - loss: 0.0786 - accuracy: 0.9773 - val_loss: 0.1522 - val_accuracy: 0.9518 - 54s/epoch - 271ms/step
Epoch 5/10
200/200 - 53s - loss: 0.0543 - accuracy: 0.9834 - val_loss: 0.1617 - val_accuracy: 0.9500 - 53s/epoch - 266ms/step
Epoch 6/10
200/200 - 55s - loss: 0.0430 - accuracy: 0.9858 - val_loss: 0.1802 - val_accuracy: 0.9475 - 55s/epoch - 276ms/step
Epoch 7/10
200/200 - 52s - loss: 0.0343 - accuracy: 0.9895 - val_loss: 0.1804 - val_accuracy: 0.9531 - 52s/epoch - 260ms/step
Epoch 8/10
200/200 - 52s - loss: 0.0279 - accuracy: 0.9914 - val_loss: 0.1427 - val_accuracy: 0.9587 - 52s/epoch - 258

In [25]:
txt_list = ["ما حال الطقس اليوم ؟","احجز لى معاد اليوم الساعة 5","كان بوم جميل جدا"," اقترح فيلم جديد لاشاهده"]
labels = ['schedule','weather','general','recommendation','greeting','thank']
answer = list()
for txt in txt_list:
    seq = tokenizer.texts_to_sequences([txt])
    padded = pad_sequences(seq, maxlen=max_length)
    pred = model.predict(padded)
    try:
        label = labels[np.argmax(pred)-1] 
        answer.append(label)
    except:
        answer.append('general')
print(answer)

1/1 [==============================] - 0s 56ms/step
['weather', 'schedule', 'general', 'recommendation']


In [26]:
model.save("models")

INFO:tensorflow:Assets written to: models\assets


INFO:tensorflow:Assets written to: models\assets


In [27]:
from tensorflow import keras

m = keras.models.load_model("models")

In [28]:
answer = list()
for txt in txt_list:
    seq = tokenizer.texts_to_sequences([txt])
    padded = pad_sequences(seq, maxlen=max_length)
    pred = m.predict(padded)
    try:
        label = labels[np.argmax(pred)-1] 
        answer.append(label)
    except:
        answer.append('general')
print(answer)

1/1 [==============================] - 0s 47ms/step
['weather', 'schedule', 'general', 'recommendation']
